# Photography lenses sales agent

## Part 1: Preprocessing

In the first part we are going to preprocess our data by splitting into chunks, and then we are going to store it in a vector database. We are going to use the OpenAI embeddings, and the Chroma vector store.

In [1]:
import os

os.environ["OPENAI_API_KEY"] = ""

The first we have to do is preprocess our product data, and store it in a vector database. The product data is a json file (lenses.json), and each object contains information about a camera lens: its name, price, shipping time, warranty, some technical specs, and a description.

In [2]:
import json

from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(
  chunk_size=500,
  chunk_overlap=50,
  separator="},"
)

with open("lenses.json", "r") as f:
    lenses_data = json.loads(f.read())

lenses = splitter.create_documents([str(lenses_data)])

We can see that the data has been split in 20 different documents, and this is exactly the number of products we have in our json file. Choosing a chunk_size of 500 and a separator of "}," helped here, as each product object is approximately 500 characters long, and the separator is the end of a product object.

In [3]:
print(len(lenses))
print(lenses[0])

20
page_content="[{'name': 'Orion 24mm f/1.4 Wide-Angle Lens', 'price': 749, 'shipping_time': '3-5 business days', 'warranty': '2 years', 'technical_details': '24mm focal length, f/1.4 maximum aperture, manual focus', 'info': 'The Orion 24mm f/1.4 Wide-Angle Lens is perfect for landscape and astrophotography. Its wide-angle view captures expansive scenes, and the fast f/1.4 aperture performs well in low light.'" metadata={}


In [4]:
lenses[5]

Document(page_content='{\'name\': \'ProFocus 135mm f/2 Portrait Lens\', \'price\': 949, \'shipping_time\': \'3-5 business days\', \'warranty\': \'2 years\', \'technical_details\': \'135mm focal length, f/2 maximum aperture, autofocus\', \'info\': "The ProFocus 135mm f/2 Portrait Lens provides a longer focal length for portrait photography, delivering a high level of clarity and detail while maintaining a pleasing background blur. It\'s ideal for outdoor portraits and studio work."', metadata={})

Now we can create a vector database, and store our documents in it.

In [5]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = OpenAIEmbeddings()
lenses_data = Chroma.from_documents(lenses, embeddings=embeddings, collection_name="lenses")

Using embedded DuckDB without persistence: data will be transient
No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


## Part 2: Creating the retrieval model

In [9]:
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain

In [7]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [10]:
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), lenses_data.as_retriever(), memory=memory)

In [12]:
qa("What lenses do you recommend for landscapes?")

{'question': 'What lenses do you recommend for landscapes?',
 'chat_history': [HumanMessage(content='What lenses do you have in store?', additional_kwargs={}, example=False),
  AIMessage(content=' We have the EagleEye 100mm f/2.8 Macro Lens, the EagleEye 60mm f/2.8 Macro Lens, the Orion 50mm f/1.4 Standard Lens, and the EagleEye 50mm f/2.8 Compact Macro Lens.', additional_kwargs={}, example=False),
  HumanMessage(content='What lenses do you recommend for landscapes?', additional_kwargs={}, example=False),
  AIMessage(content=' I would recommend the ZoomMaster 18-35mm f/1.8 Wide Zoom Lens, the WideMaster 10mm f/4 Ultra-Wide Angle Lens, the Orion 24mm f/1.4 Wide-Angle Lens, and the WideMaster 20mm f/1.8 Wide-Angle Lens. All of these lenses have wide-angle views and fast maximum apertures that are ideal for landscape photography.', additional_kwargs={}, example=False)],
 'answer': ' I would recommend the ZoomMaster 18-35mm f/1.8 Wide Zoom Lens, the WideMaster 10mm f/4 Ultra-Wide Angle Len

In [13]:
qa("And for portraits")['answer']

' I recommend the ProFocus 105mm f/2 Portrait Lens, ProFocus 135mm f/2 Portrait Lens, and ProFocus 200mm f/2.8 Portrait Lens for portraits. The EagleEye 60mm f/2.8 Macro Lens is also suitable for portraits due to its flattering perspective.'